# Data preparation




In [ ]:
PATH = "/content/drive/MyDrive/MSc /Narratives Project Nadav/analysis/data/raw"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/MSc /Narratives Project Nadav/analysis/data/raw"

storeleads-scraping-about_data_toclean.csv  storeleads-scraping-about_data_toclean_sample5.csv


In [ ]:
import os
import pandas as pd
import numpy as np

data_path = os.path.join(PATH, "storeleads-scraping-about_data_toclean_sample5.csv")
df = pd.read_csv(data_path)



# add empty row called clean text
df['clean_text'] = ''
df['purged_text'] = ''
display(df.head())

,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://crafthq.co.uk/pages/about-us,Free delivery on orders over £40Free delivery ...,0.0,7,crafthq.co.uk,crafthq.co.uk/pages/about-us,US,en,shopify,,
1,https://active-silver.co.uk/pages/about-active...,Country/regionCountry/region ...,5.0,7,active-silver.co.uk,active-silver.co.uk/pages/about-active-silver,US,en,shopify,,
2,https://smartstore24.org/pages/about-us,About Us Summer Sale - 50% OFF + FREE Shipping...,1.0,4,smartstore24.org,smartstore24.org/pages/about-us,CN,en,shopify,,
3,https://www.kbeautyshop.com.co/quienes-somos,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,1.0,17,kbeautyshop.com.co,kbeautyshop.com.co/quienes-somos,CO,es,tiendanube,,
4,https://www.infinitymarts.com.br/pages/quem-somos,Quem Somos Frete Grátis para todo o Brasil ...,1.0,13,infinitymarts.com.br,infinitymarts.com.br/pages/quem-somos,BR,pt,shopify,,


In [ ]:
!pip install langid -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import langid

def is_gibberish(text, confidence_threshold=0.9):
    """
    Returns True if 'text' is likely gibberish
    or not recognized as a valid language with high confidence.
    """
    # If it's empty or just whitespace, call it gibberish
    if not isinstance(text, str) or not text.strip():
        return True

    # langid.classify() returns (language_code, confidence)
    lang_code, conf = langid.classify(text)
    # If confidence is below threshold, consider it gibberish
    if conf < confidence_threshold:
        return True

    return False

In [ ]:
import pandas as pd

def remove_gibberish_rows(df, text_col="text", conf_thr=0.9):
    """
    Drop rows where text is recognized as gibberish
    """
    mask = []
    for text in df[text_col]:
        mask.append(not is_gibberish(text, confidence_threshold=conf_thr))
    # Keep only rows whose 'text' is recognized as valid
    return df[mask]

# Example usage:
df_clean = remove_gibberish_rows(df, text_col="text", conf_thr=0.9)


In [ ]:
print(len(df_clean))

23


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb 13 12:20:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/ModernBERT-base-zeroshot-v2.0",
    tokenizer="MoritzLaurer/ModernBERT-base-zeroshot-v2.0",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
text = """About FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club. Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner. We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession. Community focused. Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia. We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams. We don't do normal. We overdeliver. From the way we teach, to the tools we provide, we believe in standing out and doing things differently. We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods. We love good humans. We believe in treating people with respect and kindness. From strangers on the Internet, to our SRNAs and fellow colleagues, we practice the Golden Rule. We value transparency and doing the right thing, even when nobody is looking. We promise to bring inclusivity, openness and acceptance to every interaction. For the CRNA school applicant, helpful articles and resources. Check out our most popular applicant resource, The CRNA School Database. Applying to CRNA school? Our digital courses will take you further, faster. And that's why we founded The CRNA Club.    """
label_1 = "ad" # @param {"type":"string","placeholder":"ad"}
label_2 = "about us" # @param {"type":"string","placeholder":"about us"}
labels = []
labels.append(label_1)
labels.append(label_2)

import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# 4. Split text into segments (paragraphs or lines)
# Split into sentences or paragraphs:
segments = sent_tokenize(text)

print(f"Text split into {len(segments)} segments.")
# Prepare lists to store the two categories
spammy_segments = []
informative_segments = []

for seg in segments:
    # Clean up the segment (strip whitespace)
    seg = seg.strip()
    if not seg:
        continue  # skip empty lines

    # 5. Classify the segment using zero-shot classification
    result = classifier(seg, labels)

    print(f"Segment: {seg}")
    print(f"Classification result: {result['scores']}")


    # The pipeline's result["labels"] might reorder the labels by descending probability
    # so we find the index of each label in result["labels"]
    spammy_score = result["scores"][result["labels"].index(labels[0])]
    informative_score = result["scores"][result["labels"].index(labels[1])]

    # 6. Decide which category is higher
    if spammy_score > informative_score:
        spammy_segments.append(seg)
    else:
        informative_segments.append(seg)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Text split into 23 segments.
Segment: About FLASH SALE!
Classification result: [0.9038354754447937, 0.0961645096540451]
Segment: 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
Classification result: [0.9906747937202454, 0.009325189515948296]
Segment: 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
Classification result: [0.6448385715484619, 0.3551614582538605]
Segment: And that's why we founded The CRNA Club.
Classification result: [0.9546354413032532, 0.04536455497145653]
Segment: Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
Classification result: [0.5764673352241516, 0.423532634973526]
Segment: We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
Classification result: [0.9756247401237488, 0.02437526546418667]
Segment: Community focused.
Classification result: [0.8605764508247375, 0.1394

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Segment: We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
Classification result: [0.8003581166267395, 0.19964182376861572]
Segment: We don't do normal.
Classification result: [0.9980670809745789, 0.001932949642650783]
Segment: We overdeliver.
Classification result: [0.977422833442688, 0.02257719449698925]
Segment: From the way we teach, to the tools we provide, we believe in standing out and doing things differently.
Classification result: [0.9918736815452576, 0.008126316592097282]
Segment: We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods.
Classification result: [0.9782304167747498, 0.021769538521766663]
Segment: We love good humans.
Classification result: [0.9263256192207336, 0.07367438077926636]
Segment: We believe in treating people with respect and kindness.
Classification result: [0.9641472697257996, 0.035852711

In [ ]:
# 7. Print or otherwise use the results
print("===== SPAMMY SEGMENTS =====")
for s in spammy_segments:
    print(f"- {s}")

print("\n===== INFORMATIVE SEGMENTS =====")
for s in informative_segments:
    print(f"- {s}")

===== SPAMMY SEGMENTS =====
- About FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
- We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
- We promise to bring inclusivity, openness and acceptance to every interaction.
- For the CRNA school applicant, helpful articles and resources.
- Applying to CRNA school?

===== INFORMATIVE SEGMENTS =====
- And that's why we founded The CRNA Club.
- Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
- We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
- Community focused.
- Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
- We don't do norma

In [ ]:
import math
def classify_subchunk(sent_list,labels):
    """
    Returns (best_label, best_score_log) for the entire subchunk.
    best_score_log = sum of log(probabilities) for the chosen label across all sentences.
    """
    # We'll sum up the log probabilities for each label across all sentences,
    # then pick which label has the higher total.
    total_ad_log = 0.0
    total_story_log = 0.0

    for s in sent_list:
        s = s.strip()
        if not s:
            continue
        result = classifier(s, labels)
        # e.g. result["labels"] = ["story", "advertisement"] in descending score order
        p_ad = result["scores"][result["labels"].index(labels[0])]
        p_story = result["scores"][result["labels"].index(labels[1])]

        # add log probabilities
        total_ad_log += math.log(p_ad + 1e-9)
        total_story_log += math.log(p_story + 1e-9)

    # pick which label is best overall
    if total_ad_log > total_story_log:
        return (labels[0], total_ad_log)
    else:
        return (labels[1], total_story_log)

# 4) Compare no-split vs. best single-split
# -- (a) No-split: treat entire chunk as one label
#print(sentences)
a ="""25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club."""

whole_label, whole_score = classify_subchunk(a)

# -- (b) Single-split: try every boundary between sentences
best_split_score = float("-inf")
best_split = None  # store index of the best split
best_left_label = None
best_right_label = None

for w in range(1, len(sentences)):  # possible split points
    left = sentences[:w]
    right = sentences[w:]

    left_label, left_score = classify_subchunk(left,labels)
    right_label, right_score = classify_subchunk(right,labels)

    total_score = left_score + right_score

    if total_score > best_split_score:
        best_split_score = total_score
        best_split = w
        best_left_label = left_label
        best_right_label = right_label

print(f"No-split label: {whole_label}, log-score = {whole_score:.3f}")
if best_split:
    print(f"Best single-split at sentence {best_split}:")
    print(f" Left label: {best_left_label}")
    print(f" Right label: {best_right_label}")
    print(f" Total two-chunk log-score = {best_split_score:.3f}")

# 5) Decide final chunking
if best_split_score > whole_score:
    # We do the split
    final_left = sentences[:best_split]
    final_right = sentences[best_split:]

    print("\nSPLIT CHOSEN:")
    print("LEFT SUB-CHUNK (labeled as", best_left_label, "):")
    for s in final_left:
        print(" -", s)
    print("\nRIGHT SUB-CHUNK (labeled as", best_right_label, "):")
    for s in final_right:
        print(" -", s)
else:
    # Keep chunk as one
    print("\nNO SPLIT CHOSEN. The entire chunk is labeled:", whole_label)
    for s in sentences:
        print(" -", s)

TypeError: classify_subchunk() missing 1 required positional argument: 'labels'

# The DP approach

In [ ]:
#classifier = pipeline("zero-shot-classification", model="microsoft/deberta-large-mnli")


In [ ]:
segment_text_viterbi(text, labels, classifier)

s

In [ ]:
import math
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")



# 1) DOWNLOAD NLTK TOKENIZER
nltk.download("punkt_tab", quiet=True)
# 2) SET UP ZERO-SHOT CLASSIFIER
#classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 3) YOUR TEXT
text = """About FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club. Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner. We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession. Community focused. Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia. We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams. We don't do normal. We overdeliver. From the way we teach, to the tools we provide, we believe in standing out and doing things differently. We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods. We love good humans. We believe in treating people with respect and kindness. From strangers on the Internet, to our SRNAs and fellow colleagues, we practice the Golden Rule. We value transparency and doing the right thing, even when nobody is looking. We promise to bring inclusivity, openness and acceptance to every interaction. For the CRNA school applicant, helpful articles and resources. Check out our most popular applicant resource, The CRNA School Database. Applying to CRNA school? Our digital courses will take you further, faster. And that's why we founded The CRNA Club.    """

def smart_split_text(text, max_tokens=200, tokenizer=None):
    """
    1) Sentence-split the text (via SpaCy or NLTK).
    2) Accumulate sentences into chunks until you reach max_tokens.
    3) Start a new chunk.
    """
    sents = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_len = 0

    for s in sents:
        tokens = tokenizer.encode(s, add_special_tokens=False)
        if (current_len + len(tokens)) > max_tokens:
            # Start a new chunk
            if current_chunk:
                chunks.append(" ".join(current_chunk))
            current_chunk = [s]
            current_len = len(tokens)
        else:
            current_chunk.append(s)
            current_len += len(tokens)

    # Add last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


# CHOOSE YOUR LABELS
labels = ["spam", "about us"]
# (Feel free to try synonyms like "promotional" vs. "educational")
def segment_text_viterbi(text, labels, classifier, smoothness=0.5, spam_th=0.5,batch_size=32):
    """
    Uses a Viterbi-like dynamic programming approach to label segments of text.
    Low spam_th => more aggressively labeling segments as labels[1] if p(labels[0]) < spam_th.

    Parameters
    ----------
    text : str
        The input text to segment and classify.
    labels : list
        A two-element list of label names, e.g. ["AD", "ST"].
    classifier : function
        A model or pipeline that returns classification results with .scores and .labels fields.
    smoothness : float
        A penalty for switching labels from segment to segment in the Viterbi transitions.
    spam_th : float
        A threshold below which we “push” the probability away from labels[0].
        For instance, if p(labels[0]) < spam_th, we favor labels[1].

    Returns
    -------
    labeled_segments : list of tuples
        Each tuple is (segment_text, assigned_label).
    """




    # 1) Tokenize into segments
    segments = smart_split_text(text, max_tokens=200, tokenizer=tokenizer)

    # 2) Compute emission probabilities
    #    We'll store probabilities as list of (p_label0, p_label1) for each segment
    probs = [(0.0, 0.0)] * len(segments)  # pre-allocate

    # Get batch results for all segments
    batch_results = classifier(segments, labels, batch_size=batch_size)

    for i, (seg, result) in enumerate(zip(segments, batch_results)):
        seg = seg.strip()
        if not seg:
            # If the segment is empty, keep default (0.0, 0.0) or handle differently if you prefer
            continue

        # Extract the probability assigned to each label
        p_ad = result["scores"][result["labels"].index(labels[0])]  # e.g., label0
        p_st = result["scores"][result["labels"].index(labels[1])]  # e.g., label1

        # --- THRESHOLD ADJUSTMENT ---
        #
        # If p_ad < spam_th, push the distribution toward label1.
        # One simple approach is to force p_ad to be extremely small
        # so that label1 is heavily favored in the final path.
        # (You can make this more subtle if you prefer.)
        #
        if p_ad < spam_th:
            p_ad = 1e-9
            p_st = 1.0 - p_ad

        # Store
        probs[i] = (p_ad, p_st)

    # 3) Define transition cost function
    #    Returns 0.0 if same label, else -smoothness
    def transition_cost(prev_label, curr_label):
        return 0.0 if prev_label == curr_label else -smoothness

    # 4) Convert to log probabilities
    #    (avoid log(0) by adding a small epsilon)
    eps = 1e-9
    log_probs = [(math.log(ad + eps), math.log(st + eps)) for (ad, st) in probs]

    n = len(segments)

    # 5) Initialize Viterbi DP arrays
    # dp[i][0] = best log-prob. ending at segment i with label0
    # dp[i][1] = best log-prob. ending at segment i with label1
    dp = [[float('-inf')] * 2 for _ in range(n)]
    bp = [[None] * 2 for _ in range(n)]  # back-pointer

    # Base case
    dp[0][0] = log_probs[0][0]
    dp[0][1] = log_probs[0][1]

    # 6) Fill DP
    for i in range(1, n):
        for curr_label in (0, 1):
            for prev_label in (0, 1):
                score = (
                    dp[i - 1][prev_label]
                    + transition_cost(prev_label, curr_label)
                    + log_probs[i][curr_label]
                )
                if score > dp[i][curr_label]:
                    dp[i][curr_label] = score
                    bp[i][curr_label] = prev_label

    # 7) Backtrack to find best path
    final_label = 0 if dp[n - 1][0] > dp[n - 1][1] else 1
    best_path = [final_label]

    for i in range(n - 1, 0, -1):
        best_path.append(bp[i][best_path[-1]])
    best_path.reverse()

    # 8) Pair segments with assigned labels
    label_map = {0: labels[0], 1: labels[1]}
    labeled_segments = [(segments[i].strip(), label_map[best_path[i]]) for i in range(n)]

    return labeled_segments




def print_segmented_text(labeled_segments):

    ads = [seg for seg, lbl in labeled_segments if lbl == labels[0]]
    stories = [seg for seg, lbl in labeled_segments if lbl == labels[1] ]

    print("===== ADVERTISEMENT SEGMENTS =====")
    for seg in ads:
        print("-", seg)

    print("\n===== STORY SEGMENTS =====")
    for seg in stories:
        print("-", seg)

def get_switching_strings(labeled_segments):

    best_path = [seg[1] for seg in labeled_segments]
    segments = [seg[0] for seg in labeled_segments]
    """
    When best_path[i] != best_path[i+1], combine segments[i] + " " + segments[i+1].
    Returns a list of these combined strings.
    """
    switching_strings = []
    switching_labels = []
    switching_indices = []
    n = len(segments)
    if n <= 1:
        return switching_strings,switching_labels,switching_indices

    for i in range(n - 1):
        if best_path[i] != best_path[i+1]:
            combined = segments[i] + " " + segments[i+1]
            #labels
            left_label = best_path[i]
            right_label = best_path[i+1]

            switching_strings.append(combined)
            switching_labels.append((left_label, right_label))
            switching_indices.append(i)

    return switching_strings, switching_labels,switching_indices

import math

def best_split_batch(chunk, left_label, right_label, classifier, all_labels):
    """
    Find the best split for 'chunk' at each possible word boundary [1..N-1],
    forcing the left sub-chunk to 'left_label' and the right sub-chunk to 'right_label',
    but we do it in a single (or batched) classifier call to leverage GPU.

    Returns
    -------
    (best_split_idx, best_score, best_left_str, best_right_str)
      best_split_idx : int or None
      best_score : float (the maximum total_log among all splits)
      best_left_str, best_right_str : str
    """
    words = chunk.split()
    n = len(words)

    # If there's no meaningful way to split (e.g. n < 2),
    # fallback to "all chunk" on left_label. You can pick a default or re-check probabilities.
    if n < 2:
        # single call or skip. For demonstration:
        result = classifier(chunk, all_labels)
        prob = result["scores"][result["labels"].index(left_label)]
        logp = math.log(prob + 1e-9)
        return (None, logp, chunk, "")

    # ----------------------------------------------------------------
    # 1) Generate all sub-chunks:
    #    For i in [1..n-1], left_str = words[:i], right_str = words[i:].
    #    We'll gather them in a single list so we can do a single classifier() call in one or more big batches.
    # ----------------------------------------------------------------
    left_subchunks = []
    right_subchunks = []
    for i in range(1, n):
        left_subchunks.append(" ".join(words[:i]))
        right_subchunks.append(" ".join(words[i:]))

    # We'll have len(words) - 1 left_subchunks, and the same number of right_subchunks.
    # We'll combine them into a single list, so we can classify them all in one go:
    combined_subchunks = left_subchunks + right_subchunks
    # Example: we have 2*(n-1) total sub-chunks.

    # ----------------------------------------------------------------
    # 2) Classify them in one (or few) big batches on the GPU.
    #    The classifier presumably returns a list of dicts with "scores" and "labels".
    # ----------------------------------------------------------------
    results = classifier(combined_subchunks, all_labels)
    # Make sure your classifier can handle large batches or break it up if needed.

    # Now, the first (n-1) results correspond to left_subchunks,
    # the next (n-1) correspond to right_subchunks.
    left_results = results[: (n - 1)]
    right_results = results[(n - 1) :]

    # ----------------------------------------------------------------
    # 3) Compute log(prob) for the assigned label for each sub-chunk.
    #    - left_subchunks must be forced to left_label
    #    - right_subchunks must be forced to right_label
    # ----------------------------------------------------------------
    left_logs = []
    for r in left_results:
        p_left = r["scores"][r["labels"].index(left_label)]
        left_logs.append(math.log(p_left + 1e-9))

    right_logs = []
    for r in right_results:
        p_right = r["scores"][r["labels"].index(right_label)]
        right_logs.append(math.log(p_right + 1e-9))

    # ----------------------------------------------------------------
    # 4) Combine left_logs[i] + right_logs[i] to find the best split
    # ----------------------------------------------------------------
    best_score = float("-inf")
    best_split_idx = None
    best_left_str = None
    best_right_str = None

    for i in range(n - 1):
        total_log = left_logs[i] + right_logs[i]
        if total_log > best_score:
            best_score = total_log
            best_split_idx = i + 1  # i+1 words go to the left
            best_left_str = left_subchunks[i]
            best_right_str = right_subchunks[i]

    return best_split_idx, best_score, best_left_str, best_right_str


def fix_splits(
    labeled_segments,
    switching_labels,
    switching_strings,
    switching_indices,
    classifier,
    labels
):
    """
    Attempt to fix segments where a 'switch' of labels is detected mid-segment.
    This version ensures we skip trivial edge splits. We do NOT merge adjacent
    segments if they share the same label.

    Parameters
    ----------
    labeled_segments : list of tuples (segment_text, label)
    switching_labels : list of tuples (left_label, right_label)
    switching_strings : list of strings (the text chunk for each switch)
    switching_indices : list of segment indices to fix
    classifier : function to evaluate splits
    labels : [label0, label1]
    """

    # 1) Create a dictionary to store the "split replacements"
    #    for segments that need to be fixed.
    split_replacements = {}

    # 2) Fix each "switching" chunk by trying best_split()
    for i, s in enumerate(switching_strings):
        left_label, right_label = switching_labels[i]
        seg_index = switching_indices[i]  # which segment to replace

        split_idx, score = best_split_batch(s, left_label, right_label, classifier, labels)
        words = s.split()

        # If the best split is at 0 or end, skip splitting and assign full chunk to one label.
        if split_idx <= 0 or split_idx >= len(words):
            # You can pick whichever label you prefer; here's an example using left_label:
            entire_label = left_label
            split_replacements[seg_index] = [(s, entire_label)]
        else:
            # Valid split: slice the text into left & right sub-chunks
            left_str = " ".join(words[:split_idx])
            right_str = " ".join(words[split_idx:])

            split_replacements[seg_index] = [
                (left_str, left_label),
                (right_str, right_label)
            ]

    # 3) Rebuild the final segmentation.
    corrected_segments = []
    for i, (seg_text, seg_label) in enumerate(labeled_segments):
        if i in split_replacements:
            # Replace with the two (or one) corrected sub-chunks
            corrected_segments.extend(split_replacements[i])
        else:
            # Keep the original segment if no replacement
            corrected_segments.append((seg_text, seg_label))

    return corrected_segments





### all together


In [ ]:

labeled_segments = segment_text_viterbi(text, labels, classifier,smoothness=0.25)
print_segmented_text(labeled_segments)
switching_strings,switching_labels, switching_indices = get_switching_strings(labeled_segments)
corrected_segments = fix_splits(labeled_segments,switching_labels,switching_strings,switching_indices, classifier,labels)

print("\n===== FINAL CORRECTED SEGMENTATION =====")
for text_seg, label_seg in corrected_segments:
    print(f"[{label_seg}] {text_seg}")

In [ ]:
def segment_text(text, labels, classifier,smoothness=0.5,spam_th=0.3,batch_size=32):
  labeled_segments = segment_text_viterbi(text, labels, classifier,smoothness=smoothness,spam_th=spam_th,batch_size=batch_size)
  #print_segmented_text(labeled_segments)
  switching_strings,switching_labels, switching_indices = get_switching_strings(labeled_segments)

  corrected_segments = fix_splits(labeled_segments,switching_labels,switching_strings,switching_indices, classifier,labels)
  return corrected_segments

corrected_segments = segment_text(text, labels, classifier,smoothness=0.25)
print("\n===== FINAL CORRECTED SEGMENTATION =====")
for text_seg, label_seg in corrected_segments:
    print(f"[{label_seg}] {text_seg}")

# Run through the entire dataset

In [ ]:
# loop through df rows
labels = ["ad","our values"]
for i,r in df.iterrows():
  print(i)
  if i < 0:
    pass
  else:
    corrected_segments = segment_text(r["text"], labels, classifier,smoothness=0.3,spam_th=0.3,batch_size=128)
    purged_text= [text_seg for text_seg, label_seg in corrected_segments if label_seg == labels[0]]
    purged_text_str = "\n".join(purged_text)

    clean_text = [text_seg for text_seg, label_seg in corrected_segments if label_seg == labels[1]]
    clean_text_str = "\n".join(clean_text)

    # put it in df["clean_text"]
    df.at[i, 'clean_text'] = clean_text_str
    df.at[i, 'purged_text'] = purged_text_str







In [ ]:
print(len(df))
print(len(df[df['clean_text'] != '']))

In [ ]:
# display rows where clean_text column isn't empty
display(df[df['clean_text'] != ''])

for i,r in df[df['clean_text'] != ''].iterrows():
  print(f"======================clean=================")
  print(r["clean_text"])
  print(f"======================purged=================")
  print(r["purged_text"])


In [ ]:
# 2) Write desired output to a text file
with open("output.txt", "w", encoding="utf-8") as f:
    for i, r in df[df["clean_text"] != ""].iterrows():
        f.write(f"=======================================\n\t\t\t\t\t\tsample{i}\n=======================================")
        f.write("======================clean=================\n")
        f.write(r["clean_text"] + "\n")  # Already contains \n between segments
        f.write("======================purged=================\n")
        f.write(r["purged_text"] + "\n") # Already contains \n between segments
        f.write(r["text"] + "\n\n")      # Original text

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import math

nltk.download("punkt", quiet=True)
labels = ["spam, ad, ", "our story, our values"]

def safe_sent_tokenize(text):
    """
    Attempt to tokenize text into sentences; if text isn't a valid string,
    catch the TypeError and return an empty list to avoid breaking the code.
    """
    try:
        return sent_tokenize(text)
    except (TypeError, AttributeError):
        # TypeError -> text is float or None
        # AttributeError -> text might be None or weird object
        return []

def batch_classify_segments(df, text_col, labels, classifier, spam_th=0.5, batch_size=32):
    """
    1) Tokenize each row's text by splitting on '.' (dumbly).
    2) Classify all segments in big batches with threshold logic.
    3) Rebuild 'clean_text' (label=1) and 'purged_text' (label=0).
    """
    # (row_idx, segment_text)
    segments = []
    # We'll store how many segments each row has, to regroup them later
    segment_counts = []

    # A) "Dumb" tokenization on dots:
    for row_i, row in df.iterrows():
        text_val = row[text_col]
        # Instead of safe_sent_tokenize(text_val), do:
        if isinstance(text_val, str):
            sents = [chunk.strip() for chunk in text_val.split(".") if chunk.strip()]
        else:
            sents = []  # handle non-string as empty

        segment_counts.append(len(sents))
        for s in sents:
            segments.append((row_i, s))

    if not segments:
        # No valid segments at all
        df["clean_text"] = ""
        df["purged_text"] = ""
        return df

    # B) Classify all segments with threshold logic
    all_segment_texts = [seg_text for (_row_i, seg_text) in segments]
    assigned_labels = classify_batch_with_threshold(
        segment_list=all_segment_texts,
        labels=labels,
        classifier=classifier,
        spam_th=spam_th,
        batch_size=batch_size
    )

    # C) Rebuild clean_text / purged_text
    df["clean_text"] = ""
    df["purged_text"] = ""

    position = 0
    for row_i, row in df.iterrows():
        seg_count = segment_counts[row_i]
        row_segments = segments[position : position + seg_count]
        row_assigned = assigned_labels[position : position + seg_count]
        position += seg_count

        purged_parts = []
        clean_parts = []
        for (seg_row_i, seg_text), lbl in zip(row_segments, row_assigned):
            if lbl == 0:
                purged_parts.append(seg_text)
            else:
                clean_parts.append(seg_text)

        df.at[row_i, "purged_text"] = " \n".join(purged_parts)
        df.at[row_i, "clean_text"]  = " \n".join(clean_parts)

    return df


def classify_batch_with_threshold(segment_list, labels, classifier, spam_th=0.5, batch_size=32):
    """
    1) Classify all segments in one or more large batches.
    2) Apply threshold logic: if p_label0 < spam_th => forcibly push it to label1.
    3) Return assigned labels (0 or 1).

    segment_list : list of str
    labels       : [label0, label1]
    classifier   : function that takes (list_of_strings, label_list, batch_size=...)
                   and returns list of dicts with "scores", "labels"
    spam_th      : threshold; if p_label0 < spam_th => p_label0=1e-9 => label1 is forced
    """
    results = []
    # We'll store the final assigned labels as integers (0 or 1) in the same order as segment_list.
    assigned_labels = []

    # Step A: batch inference
    for start in range(0, len(segment_list), batch_size):
        batch = segment_list[start : start + batch_size]
        batch_outputs = classifier(batch, labels, batch_size=batch_size)

        # Step B: threshold logic, pick label
        for res in batch_outputs:
            # Probability of label0, label1
            idx0 = res["labels"].index(labels[0])
            idx1 = res["labels"].index(labels[1])
            p0 = res["scores"][idx0]
            p1 = res["scores"][idx1]

            # If p0 < spam_th, force p0 near zero => label1 is chosen
            if p0 < spam_th:
                p0 = 1e-9
                p1 = 1.0 - p0

            # Decide label by max probability
            if p0 > p1:
                assigned_labels.append(0)
            else:
                assigned_labels.append(1)

    return assigned_labels

def batch_classify_segments(df, text_col, labels, classifier, spam_th=0.5, batch_size=32):
    """
    1) Tokenize each row's text by splitting on '.' (dumbly).
    2) Classify all segments in big batches with threshold logic.
    3) Rebuild 'clean_text' (label=1) and 'purged_text' (label=0).
    """
    # (row_idx, segment_text)
    segments = []
    # We'll store how many segments each row has, to regroup them later
    segment_counts = []

    # A) "Dumb" tokenization on dots:
    for row_i, row in df.iterrows():
        text_val = row[text_col]
        # Instead of safe_sent_tokenize(text_val), do:
        if isinstance(text_val, str):
            sents = [chunk.strip() for chunk in text_val.split(".") if chunk.strip()]
        else:
            sents = []  # handle non-string as empty

        segment_counts.append(len(sents))
        for s in sents:
            segments.append((row_i, s))

    if not segments:
        # No valid segments at all
        df["clean_text"] = ""
        df["purged_text"] = ""
        return df

    # B) Classify all segments with threshold logic
    all_segment_texts = [seg_text for (_row_i, seg_text) in segments]
    assigned_labels = classify_batch_with_threshold(
        segment_list=all_segment_texts,
        labels=labels,
        classifier=classifier,
        spam_th=spam_th,
        batch_size=batch_size
    )

    # C) Rebuild clean_text / purged_text
    df["clean_text"] = ""
    df["purged_text"] = ""

    position = 0
    for row_i, row in df.iterrows():
        seg_count = segment_counts[row_i]
        row_segments = segments[position : position + seg_count]
        row_assigned = assigned_labels[position : position + seg_count]
        position += seg_count

        purged_parts = []
        clean_parts = []
        for (seg_row_i, seg_text), lbl in zip(row_segments, row_assigned):
            if lbl == 0:
                purged_parts.append(seg_text)
            else:
                clean_parts.append(seg_text)

        df.at[row_i, "purged_text"] = " \n".join(purged_parts)
        df.at[row_i, "clean_text"]  = " \n".join(clean_parts)

    return df




df2 = df.copy().head(4000)
df2 = df2.reset_index(drop=True)

df2 = batch_classify_segments(df2, text_col="text", labels=labels, classifier=classifier, batch_size=2048,spam_th=0.35)




In [ ]:
import gc
gc.collect()


1153

In [ ]:
PYTORCH_CUDA_ALLOC_CONF=True
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 3            |        cudaMalloc retries: 5         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      | 612891 KiB |  39888 MiB |  19177 GiB |  19176 GiB |\n|       from large pool | 612736 KiB |  39887 MiB |  19175 GiB |  19174 GiB |\n|       from small pool |    155 KiB |      1 MiB |      1 GiB |      1 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         | 612891 KiB |  39888 MiB |  19177 GiB |  19176 GiB |\n|       from large pool | 612736 KiB |  39887 MiB |

In [ ]:

# display rows where clean_text column isn't empty
display(df2[df2['clean_text'] != ''].head(10))

for i,r in df2[df2['clean_text'] != ''].head(10).iterrows():
  print(f"======================clean=================")
  print(r["clean_text"])
  print(f"======================purged=================")
  print(r["purged_text"])


,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://helios.vn/pages/about-helios,Giỏ hàng của bạn đang trống!CHÚNG TÔI LÀ HELIO...,2.0,11,helios.vn,helios.vn/pages/about-helios,US,en,shopify,Hãy để phụ kiện là điểm nhấn cuối cùng cho out...,Giỏ hàng của bạn đang trống!CHÚNG TÔI LÀ HELIO...
2,https://borakisgreekfood.co.uk/pages/about-us,About Us We are a family business based in Sou...,2.0,15,borakisgreekfood.co.uk,borakisgreekfood.co.uk/pages/about-us,NL,en,shopify,About Us We are a family business based in Sou...,"Customers began to ask for recipe ideas, so I ..."
3,https://lanienterprise.com/pages/about-store,About Store Have an account? ...,1.0,10,lanienterprise.com,lanienterprise.com/pages/about-store,US,en,shopify,Loading \nR 0 \n00 ZAR W...,About Store Have an account? ...
4,https://thexhaus.com/pages/about,About NEW COLLECTION OUT NOW The inspirat...,1.0,9,thexhaus.com,thexhaus.com/pages/about,GB,en,shopify,About NEW COLLECTION OUT NOW The inspirat...,Each piece makes a bold statement \nIt was a g...
5,https://culturecanyon.com/pages/about-culture-...,Culture Canyon • NO RUSH SHIPPING IS CURRENTLY...,0.0,9,culturecanyon.com,culturecanyon.com/pages/about-culture-canyon-a...,US,en,shopify,"Mosts items ship within 3-5 business days, how...",Culture Canyon • NO RUSH SHIPPING IS CURRENTLY...
6,https://beebaltic.com/pages/about,Our Story Free shipping on all orders over £95...,3.0,14,beebaltic.com,beebaltic.com/pages/about,US,en,shopify,Our Story Free shipping on all orders over £95...,While many people believe that this is a sign ...
7,https://shop.lavendaire.com/pages/about,Your cart is emptyFree US shipping over $90 |...,1.0,13,shop.lavendaire.com,shop.lavendaire.com/pages/about,US,en,shopify,Our goal is to empower you with tools and reso...,Your cart is emptyFree US shipping over $90 |...
8,https://4lovebirds.com/pages/about-us,Have an account? Log in to c...,18.0,81,4lovebirds.com,4lovebirds.com/pages/about-us,ES,en,shopify,Loading \n$0 \n😷🎄4Lovebirds was created by us ...,Have an account? Log in to c...
9,https://vitality-supplements.co.uk/pages/about-us,Have an account? Log in to c...,5.0,10,vitality-supplements.co.uk,vitality-supplements.co.uk/pages/about-us,GB,en,shopify,Loading \n£0,Have an account? Log in to c...
10,https://esfa-store.com/pages/nosotros,Nosotros Necesitas ayuda?NosotrosESFA inició a...,1.0,15,esfa-store.com,esfa-store.com/pages/nosotros,PE,es,shopify,Nosotros Necesitas ayuda?NosotrosESFA inició a...,comCompra SeguraAmbiente seguro para pagament...


======================clean=================
Hãy để phụ kiện là điểm nhấn cuối cùng cho outfit của bạn!     NAM TÍNH VÀ MẠNH MẼTÁO BẠO VÀ ĐỘC NHẤTTRẦM ỔNHELIOS - CHÚNG TÔI LÀ AI?         Thành lập từ năm 2016, Helios là thương hiệu phụ kiện thời trang mang đến những chế tác thủ công độc nhất và khác biệt 
Mạnh mẽ! Táo bạo! Độc nhất!                  Đây là tinh thần cốt lõi Helios luôn hướng đến và thể hiện qua mỗi sản phẩm 
Lấy cảm hứng từ những câu chuyện xoay quanh cuộc sống người đàn ông, chúng tôi làm nên các chế tác độc đáo, mang dấu ấn riêng và không thể gặp ở bất cứ đâu 
ĐIỀU GÌ TẠO NÊN HELIOS?         CHUẨN                  Mang trong mình niềm đam mê với trang sức thủ công, Helios không ngừng lắng nghe và hoàn thiện mình qua từng ngày, từ đó đem tới cho khách hàng trải nghiệm, dịch vụ và sản phẩm CHUẨN cam kết 
HÀNH TRÌNH CỦA HELIOS TRONG 5 NĂM TỚI?         Những năm trở lại đây, ngành hàng thủ công tại Việt Nam đã bắt đầu vươn ra thị trường nước ngoài 
Tuy nhiên, so với các 

In [ ]:
#print 7th
print(df.iloc[7]['text'])

In [ ]:
display(df.head(10))

In [ ]:
print(len(df2))

4000


In [ ]:
df2.to_csv("output.csv", index=False, encoding="utf-8-sig")
print("Saved to cleaned_output.csv with UTF-8 BOM, which can help Excel read it.")


Saved to cleaned_output.csv with UTF-8 BOM, which can help Excel read it.


In [ ]:
#display saved output.csv
df3 = pd.read_csv("output.csv")
display(df3.head(10))

,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://helios.vn/pages/about-helios,Giỏ hàng của bạn đang trống!CHÚNG TÔI LÀ HELIO...,2.0,11,helios.vn,helios.vn/pages/about-helios,US,en,shopify,Hãy để phụ kiện là điểm nhấn cuối cùng cho out...,Giỏ hàng của bạn đang trống!CHÚNG TÔI LÀ HELIO...
1,https://grakaibrand.com/pages/about-us,About Us FREE DOMESTIC SHIPPING ON ALL ORDERS ...,1.0,2,grakaibrand.com,grakaibrand.com/pages/about-us,CA,en,shopify,NaN,About Us FREE DOMESTIC SHIPPING ON ALL ORDERS ...
2,https://borakisgreekfood.co.uk/pages/about-us,About Us We are a family business based in Sou...,2.0,15,borakisgreekfood.co.uk,borakisgreekfood.co.uk/pages/about-us,NL,en,shopify,About Us We are a family business based in Sou...,"Customers began to ask for recipe ideas, so I ..."
3,https://lanienterprise.com/pages/about-store,About Store Have an account? ...,1.0,10,lanienterprise.com,lanienterprise.com/pages/about-store,US,en,shopify,Loading \nR 0 \n00 ZAR W...,About Store Have an account? ...
4,https://thexhaus.com/pages/about,About NEW COLLECTION OUT NOW The inspirat...,1.0,9,thexhaus.com,thexhaus.com/pages/about,GB,en,shopify,About NEW COLLECTION OUT NOW The inspirat...,Each piece makes a bold statement \nIt was a g...
5,https://culturecanyon.com/pages/about-culture-...,Culture Canyon • NO RUSH SHIPPING IS CURRENTLY...,0.0,9,culturecanyon.com,culturecanyon.com/pages/about-culture-canyon-a...,US,en,shopify,"Mosts items ship within 3-5 business days, how...",Culture Canyon • NO RUSH SHIPPING IS CURRENTLY...
6,https://beebaltic.com/pages/about,Our Story Free shipping on all orders over £95...,3.0,14,beebaltic.com,beebaltic.com/pages/about,US,en,shopify,Our Story Free shipping on all orders over £95...,While many people believe that this is a sign ...
7,https://shop.lavendaire.com/pages/about,Your cart is emptyFree US shipping over $90 |...,1.0,13,shop.lavendaire.com,shop.lavendaire.com/pages/about,US,en,shopify,Our goal is to empower you with tools and reso...,Your cart is emptyFree US shipping over $90 |...
8,https://4lovebirds.com/pages/about-us,Have an account? Log in to c...,18.0,81,4lovebirds.com,4lovebirds.com/pages/about-us,ES,en,shopify,Loading \n$0 \n😷🎄4Lovebirds was created by us ...,Have an account? Log in to c...
9,https://vitality-supplements.co.uk/pages/about-us,Have an account? Log in to c...,5.0,10,vitality-supplements.co.uk,vitality-supplements.co.uk/pages/about-us,GB,en,shopify,Loading \n£0,Have an account? Log in to c...


In [ ]:
# read batch batch_6808a851b68481908d8444dc940ead75
import openai
client = openai.